In [67]:
import pandas as pd

file_path = "./output3.csv"
df = pd.read_csv(file_path)

In [68]:
df.drop(columns=['address'], inplace=True)
df.dropna(subset=['city'], inplace=True)

In [69]:
apartment_count = 0
house_count = 0
for index, row in df.iterrows():
    # Check the value of the 'apartment_or_house' column
    if row['apartment_or_house'] == "house":
        # If it's 'house', replace it with 1
        df.at[index, 'apartment_or_house'] = 1
        house_count += 1
    elif row['apartment_or_house'] == "apartment":
        # If it's 'apartment', replace it with 0
        df.at[index, 'apartment_or_house'] = 0
        apartment_count += 1
    

print(apartment_count)
print(house_count)

53106
13828


In [30]:
df.info()
df['city'].unique()

<class 'pandas.core.frame.DataFrame'>
Index: 66934 entries, 0 to 66934
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   city                66934 non-null  object
 1   price               66934 non-null  object
 2   rooms               66934 non-null  object
 3   bathrooms           66934 non-null  object
 4   sqft                66934 non-null  object
 5   apartment_or_house  66934 non-null  object
dtypes: object(6)
memory usage: 5.6+ MB


array(['Woodbridge, VA', 'Puyallup, WA', 'Coppell, TX', 'Annandale, VA',
       'Peachtree Corners, GA', 'Waxahachie, TX', 'Annapolis, MD',
       'Lake Oswego, OR', 'Romeoville, IL', 'Hurst, TX', 'Lynnwood, WA',
       'Fort Lee, NJ', 'Issaquah, WA', 'Wheeling, IL',
       'Winter Springs, FL', 'Northglenn, CO', 'Tucker, GA',
       'Richfield, MN', 'Farmers Branch, TX', 'Norristown, PA',
       'West Hollywood, CA', 'Douglasville, GA', 'Watertown Town, MA',
       'Woodstock, GA', 'Fair Oaks, VA', 'Owings Mills, MD',
       'Panama City, FL', 'Englewood, CO', 'Kennesaw, GA',
       'Wheat Ridge, CO', 'Duluth, GA', 'Dania Beach, FL', 'Garner, NC',
       'Norwood, MA', 'Newark, DE', 'Parkville, MD', 'Lawrenceville, GA',
       'Chamblee, GA', 'Morrisville, NC', 'Fridley, MN', 'Matthews, NC',
       'Laurel, MD', 'Carney, MD', 'McDonough, GA', 'Casselberry, FL',
       'Stockbridge, GA', 'Saratoga Springs, NY', 'Short Pump, VA',
       'Fredericksburg, VA', 'Maryland Heights, MO', 'Tua

In [70]:
#Label Binarizer for States
from sklearn.preprocessing import LabelBinarizer

city_column = df["city"].to_numpy()

label_binarizer = LabelBinarizer()
label_binarizer.fit(city_column)

binarized_states = label_binarizer.transform(city_column)
binarized_city_df = pd.DataFrame(binarized_states, columns=label_binarizer.classes_)

binarized_city_df.head()


df = pd.concat([df, binarized_city_df], axis=1)
df.info()
# df['city'] = binarized_states
# print(df['city'])


<class 'pandas.core.frame.DataFrame'>
Index: 66935 entries, 0 to 64501
Columns: 862 entries, city to city
dtypes: float64(856), object(6)
memory usage: 440.7+ MB


In [71]:
# Convert NumPy array to pandas DataFrame
binarized_states_df = pd.DataFrame(binarized_states)

# Print the DataFrame
print(binarized_states_df)

       0    1    2    3    4    5    6    7    8    9    ...  846  847  848  \
0        0    0    0    0    0    0    0    0    0    0  ...    1    0    0   
1        0    0    0    0    0    0    0    0    0    0  ...    1    0    0   
2        0    0    0    0    0    0    0    0    0    0  ...    1    0    0   
3        0    0    0    0    0    0    0    0    0    0  ...    1    0    0   
4        0    0    0    0    0    0    0    0    0    0  ...    1    0    0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
66929    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
66930    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
66931    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
66932    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
66933    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   

       849  850  851  852  853  854  855  
0       

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66935 entries, 0 to 64501
Columns: 862 entries, city to city
dtypes: float64(856), object(6)
memory usage: 440.7+ MB


In [72]:
#Need to drop anything that isn't call for rent
df = df[pd.to_numeric(df['price'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['rooms'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['bathrooms'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['sqft'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['apartment_or_house'], errors='coerce').notnull()]

# Convert 'price' column to numeric
df['price'] = pd.to_numeric(df['price'])
df['rooms'] = pd.to_numeric(df['rooms'])
df['bathrooms'] = pd.to_numeric(df['bathrooms'])
df['sqft'] = pd.to_numeric(df['sqft'])
df["apartment_or_house"] = pd.to_numeric(df["apartment_or_house"])

In [48]:
print(df.info())

counts = df['apartment_or_house'].value_counts()
print(counts)

<class 'pandas.core.frame.DataFrame'>
Index: 66821 entries, 0 to 66934
Columns: 862 entries, city to city
dtypes: float64(858), int64(3), object(1)
memory usage: 440.0+ MB
None
apartment_or_house
0    52995
1    13826
Name: count, dtype: int64


In [74]:

df = df.drop(columns=['city', 'city'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66821 entries, 0 to 66934
Columns: 860 entries, price to Yukon, OK
dtypes: float64(857), int64(3)
memory usage: 438.9 MB


In [53]:
# Calculate Pearson correlation coefficients
correlation_matrix = df.corr().abs()

# Extract correlations with the target variable
feature_target_correlations = correlation_matrix['price'].drop('price')

# Print feature-target correlations
print("Feature-Target Correlations:")
print(feature_target_correlations)

Feature-Target Correlations:
rooms                 0.373022
bathrooms             0.419756
sqft                  0.185947
apartment_or_house    0.531467
Abilene, TX           0.009561
                        ...   
Woodstock, GA         0.009634
Worcester, MA         0.009698
Worthington, OH       0.008004
Yonkers, NY           0.008142
Yukon, OK             0.011068
Name: price, Length: 859, dtype: float64


In [22]:
from sklearn.preprocessing import LabelEncoder


# for col in ['city', 'apartment_or_house']:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col].astype(str))
#     label_encoders[col] = le
#     print(label_encoders)



print(df['city'].unique())



NameError: name 'data' is not defined

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import pandas as pd

df = df.dropna()
# Handle missing values if any
# imputer = SimpleImputer(strategy='mean')
# df['sqft'] = imputer.fit_transform(df['sqft'].values.reshape(-1, 1))

# Split data into train and test sets
X = df.drop(columns=['price'])  # Features
print(X.info())
print(X.fillna(0))
y = df['price']  # Target variable



<class 'pandas.core.frame.DataFrame'>
Index: 66820 entries, 0 to 66933
Columns: 859 entries, rooms to Yukon, OK
dtypes: float64(857), int64(2)
memory usage: 438.4 MB
None
       rooms  bathrooms    sqft  apartment_or_house  Abilene, TX  Addison, TX  \
0          1        1.0   757.0                   0          0.0          0.0   
1          2        2.0  1218.0                   0          0.0          0.0   
2          1        1.0   777.0                   0          0.0          0.0   
3          2        2.0  1719.0                   0          0.0          0.0   
4          3        2.0  1719.0                   0          0.0          0.0   
...      ...        ...     ...                 ...          ...          ...   
66929      4        4.0  1482.0                   1          0.0          0.0   
66930      1        1.0  1575.0                   1          0.0          0.0   
66931      2        2.0  1837.0                   1          0.0          0.0   
66932      3       

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Gaussian NB model
model = GaussianNB()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.06165818617180485


In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Split features and target variable
X = df[['city', 'rooms', 'bathrooms', 'sqft', 'apartment_or_house']]
y = df['price']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Epoch 1/50
1339/1339 [==============================] - 5s 3ms/step - loss: 2425617.5000 - val_loss: 1631576.7500
Epoch 2/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1600854.8750 - val_loss: 1589783.3750
Epoch 3/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1576215.7500 - val_loss: 1578907.7500
Epoch 4/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1567890.3750 - val_loss: 1572773.6250
Epoch 5/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1563805.2500 - val_loss: 1571341.2500
Epoch 6/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1561428.6250 - val_loss: 1569752.1250
Epoch 7/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1559805.2500 - val_loss: 1570029.5000
Epoch 8/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1559265.7500 - val_loss: 1568459.8750
Epoch 9/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1558110.7500